In [1]:
import datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import requests
import codecs
import os

wdir = "C:/Users/MarekSedlacek/Downloads/W251_Project/"

In [168]:
#os.chdir(wdir+"SilkRoad2/"+str(year)+"-"+str(month)+"-"+str(day)+"/products")
os.chdir(wdir+"SilkRoad2/2014-06-02/items")

#Reads in the HTML file
data = open("0-5-grams-pure-6-apb-powder.html",'r').read()
soup = BeautifulSoup(data)

In [26]:
os.chdir(wdir+"SilkRoad2")
os.getcwd()
#print next(os.walk(os.getcwd()))[1]
for X in next(os.walk(os.getcwd()))[1]:
    os.chdir(os.getcwd()+"\\"+X+"\items")
    print os.getcwd()
    print X
    os.chdir(wdir+"SilkRoad2")
    

C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-02\items
2014-06-02
C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-11\items
2014-06-11


In [181]:
non_decimal = re.compile(r'[^\d.]+')

sidebar = soup.find("div",attrs={"class":"sidebar"})
Category = sidebar.find("a").get_text()


body = soup.find("div",attrs={"class":"body"})
title = body.find("h2").get_text()
price = float(non_decimal.sub("",body.find("div",attrs={"class":"price_big"}).get_text()))
price_dollar = price*658.79
vendor = body.find("a").get_text()
S1 = body.find_all("p")
if S1[0].get_text()=="":
    S1 = S1[1].get_text().replace("ships from: ","").replace("ships to: ","")
else: S1 = S1[0].get_text().replace("ships from: ","").replace("ships to: ","")
Ship_From = S1.splitlines()[1]
Ship_To = S1.splitlines()[2]

In [120]:
col_names = ["Title","Category","Price","Price Dollar","Vendor","Ships From", "Ships To"]
df = pd.DataFrame(columns = col_names)
new = np.array([title,Category,price,price_dollar,vendor,Ship_From,Ship_To])
df = df.append(pd.DataFrame(new, index=col_names).transpose())

In [186]:
col_names = ["Title","Category","Price","Price Dollar","Vendor","Ships From", "Ships To"]
df = pd.DataFrame(columns = col_names)

In [39]:
os.chdir(wdir+"SilkRoad2")
col_names = ["Title","Category","Price","Price Dollar","Vendor","Ships From", "Ships To"]
df = pd.DataFrame(columns = col_names)

for X in next(os.walk(os.getcwd()))[1]:
    os.chdir(os.getcwd()+"\\"+X+"\items")
    print os.getcwd()
    print X
    
    c = 0
    for i in os.listdir(os.getcwd()):        
        try:
            if i.endswith(".html"):
                c+=1
            #Reads in the HTML file
                data = open(i,'r').read()
                soup = BeautifulSoup(data)

                non_decimal = re.compile(r'[^\d.]+')

                sidebar = soup.find("div",attrs={"class":"sidebar"})
                Category = sidebar.find("a").get_text()


                body = soup.find("div",attrs={"class":"body"})
                title = body.find("h2").get_text()
                price = float(non_decimal.sub("",body.find("div",attrs={"class":"price_big"}).get_text()))
                price_dollar = price*658.79
                vendor = body.find("a").get_text()
                S1 = body.find_all("p")
                if S1[0].get_text()=="":
                    S1 = S1[1].get_text().replace("ships from: ","").replace("ships to: ","")
                else: S1 = S1[0].get_text().replace("ships from: ","").replace("ships to: ","")
                Ship_From = S1.splitlines()[1]
                Ship_To = S1.splitlines()[2]

                col_names = ["Title","Category","Price","Price Dollar","Vendor","Ships From", "Ships To","Date"]
                new = np.array([title,Category,price,price_dollar,vendor,Ship_From,Ship_To,X])
                df = df.append(pd.DataFrame(new, index=col_names).transpose())
                
                row = {"Date":X,"Title":title,"Category":Category,"Price":price,"price_dollar":price_dollar,"Vendor":vendor,"Ships_From":Ship_From,"Ships_To":Ship_To}

                if c%100==0: print c

            else:
                continue

        except:
            print "File " + i + " has failed"
            
    os.chdir(wdir+"SilkRoad2")
        

C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-02\items
2014-06-02
100
200
300
File raise-your-flag-sr-branded-flags-look-great-when-flying-from-your-totem%3Fvendor_feedback_page=3.html has failed
File rolex-daydate-president-full-yellow-gold-aaa%3Fvendor_feedback_page=26.html has failed
400
File victoria-s-secret-bikini-replica-leopard-black-white-vs7-15.html has failed
File yeah-we-ran-out-of-drugs-how-about-some-stickers-3-5-x-6-sr-in-white%3Fvendor_feedback_page=48.html has failed
File yeah-we-ran-out-of-drugs-how-about-some-stickers-4-x3-warhol-snowden%3Fvendor_feedback_page=48.html has failed
C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-11\items
2014-06-11
100
200
300
400
500
600
700


In [50]:
df = df.drop_duplicates(subset=['Vendor',"Title","Date"],keep="first")
df.index = range(df.shape[0])
df

,Category,Date,Price,Price Dollar,Ships From,Ships To,Title,Vendor
0,Alcohol,2014-06-02,0.069872,46.03097488,Australia,Australia,0.1 Grams (1 point) of DMT sourced from Brazil...,DimitryTOO
1,Stimulants,2014-06-02,0.100882,66.46005278,Australia,Australia,0.1g (One Point) Crystal Methamphetamine / ICE,Dekay
2,Stimulants,2014-06-02,0.0481,31.687799,Germany,Undeclared,0.25g 2-DPMP,PurityStandard
3,Cocaine,2014-06-02,0.098823,65.10360417,Germany,Worldwide,0.5 gr. Interways Cocaine ** Disco Shit**,The Interways
4,Stimulants,2014-06-02,0.08294,54.6400426,Canada,Worldwide,0.5 grams crystal meth (ice) high quality,Scaptain
5,Stimulants,2014-06-02,0.047312,31.16867248,United States,United States & Canada,0.5 Grams Pure 6-APB Powder,geoffreygiraffe
6,Concentrates,2014-06-02,0.705469,464.75592251,United States,United States,1/2 Oz of shatter / sap BHO cannabis concentra...,bhokingz420
7,Ecstasy,2014-06-02,0.138931,91.52635349,Spain,Worldwide except Australia,"++++1,5 GRAMS+++[4-MEC VERY BIG CRISTALS HIGHE...",chemicals_spain
8,Alcohol,2014-06-02,0.061098,40.25075142,Undeclared,Worldwide except Australia,1 gr Refined Afghan Opium. High Quality Opium....,number_five
9,Cocaine,2014-06-02,0.167758,110.51729282,Germany,Worldwide,1 gram of High Quality Cocaine,Sukey
